# DLO-JZ Imagnet Race - Jour 3 

![race](./images/F1.png)


Le but de ce TP est de paramétrer l'entraînement pour participer à la course Imagenet Racing.

Les *job* de chaque participant durant environ 30 minutes, s'exécuteront pendant la nuit. Les résultats seront commentés le lendemain.

Les cellules dans ce *notebook* ne sont pas prévues pour être modifiées, sauf rares exceptions indiquées dans les commentaires. Les TP se feront en modifiant le code `dlojz_imagenetrace.py`.
 
*Notebook rédigé par l'équipe assistance IA de l'IDRIS, juin 2023*


------------------------

### Environnement de calcul

Un module PyTorch doit avoir été chargé pour le bon fonctionnement de ce Notebook. **Nécessairement**, le module `pytorch-gpu/py3/1.11.0` :

In [ ]:
!module list

Les fonctions *python* de gestion de queue SLURM dévelopées par l'IDRIS et les fonctions dédiées à la formation DLO-JZ sont à importer.

Le module d'environnement pour les *jobs* et la taille des images sont fixés pour ce *notebook*.

**TODO :** choisir un *pseudonyme* (maximum 5 caractères) pour vous différencier dans la queue SLURM et dans les outils collaboratifs pendant la formation et la compétition.

In [ ]:
from idr_pytools import display_slurm_queue, gpu_jobs_submitter, search_log
from dlojz_tools import controle_technique, compare, GPU_underthehood, plot_accuracy, lrfind_plot, imagenet_starter
MODULE = 'pytorch-gpu/py3/1.11.0'
account = 'for@v100'
# TODO
name = 'pseudo'   ## Pseudonyme à choisir

Creation d'un repertoire `checkpoints` si cela n'a pas déjà été fait.

In [ ]:
!mkdir checkpoints

------------------------------------

### Gestion de la queue SLURM

Cette partie permet d'afficher et de gérer la queue SLURM.

Pour afficher toute la queue *utilisateur* :

In [ ]:
display_slurm_queue(name)

**Remarque**: Cette fonction utilisée plusieurs fois dans ce *notebook* permet d'afficher la queue de manière dynamique, rafraichie toutes les 5 secondes. Cependant elle ne s'arrête que lorsque la queue est vide. Si vous désirez reprendre la main sur le *notebook*, il vous suffira d'arrêter manuellement la cellule avec le bouton *stop*. Cela a bien sûr aucun impact sur le *scheduler* SLURM. Les *jobs* ne seront pas arrêtés.

Si vous voulez arrêter des *jobs* dans la queue:
* Annuler tous vos *jobs* dans la queue (décommenter la ligne suivante) : `!scancel -u $USER`
* Annuler un *job* dans votre queue (décommenter la ligne suivante et ajouter le numéro du *job* à la fin de la ligne)


In [ ]:
#!scancel -u $USER

------------------------------------

### Debug

Cette partie *debug* permet d'afficher les fichiers de sortie et les fichiers d'erreur du *job*.

Il est nécessaire dans la cellule suivante (en décommentant) d'indiquer le *jobid* correspondant sous le format suivant.

***Remarque*** : dans ce notebook, lorsque vous soumettrez un *job*, vous recevrez en retour le numéro du job dans le format suivant : `jobid = ['123456']`. La cellule ci-dessous peut ainsi être facilement actualisée."

In [ ]:
jobid = ['2088207']

Fichier de sortie :

In [ ]:
%cat {search_log(contains=jobid[0])[0]}

Fichier d'erreur :

In [ ]:
%cat {search_log(contains=jobid[0], with_err=True)['stderr'][0]}

--------------

### Différence entre deux scripts

Pour le *debug* ou pour comparer son code avec les solutions mises à disposition, la fonction suivante permet d'afficher une page html contenant un différentiel de fichiers texte.

In [ ]:
s1 = "dlojz_imagenetrace.py"
s2 = "./solutions/dlojz2_1.py"
compare(s1, s2)

Voir le résultat du différentiel de fichiers sur la page suivante (attention au spoil !) :

[compare.html](compare.html)

----------------------

## Préparation de votre machine

> “Entre trop et trop peu est la juste mesure.” -- Gilles de Noyers

In [ ]:
from dlojz_tools import plot_accuracy, imagenet_starter, plot_time, turbo_profiler

![car](./images/noun-car-repair-32305.png)


### 1. Choix des hyper paramètres de l'apprentissage



**TODO :** Choisir la *taille de batch par GPU*  `batch_size` et la *taille d'image* `image_size` permettant d'avoir un bon équilibre (d'après votre intuition) entre une taille d'image suffisante et un nombre d'*epochs* suffisant.

* Vous devez choisir:
  * la taille des images pour l'apprentissage `image_size`
  * la *taille de batch par GPU*  `batch_size`

Le nombre d'*epochs* auquel vous avez le droit dépend du *Throughput* mesuré pendant le test. Il faudra regarder la dernière ligne du test `Eligible to run X epochs` pour connaître cette mesure.

**TODO :** 
* Veuillez choisir l'**optimizer** que vous souhaitez appliquer selon ce que l'on a vu lors du TP sur les *optimizer* et l'implémenter dans `dlojz_imagenetrace.py` si vous voulez autre chose que *SGD*.
* (Optionnel) vous pouvez aussi choisir d'implémenter dans `dlojz_imagenetrace.py` un autre LR Scheduler que le `OneCycle` de `torch` présent actuellement dans le code.
* Vous devez ensuite choisir :
  * le learning rate maximum `lr`
  * la valeur de *weight decay* `weight_decay`
  * optionnellement changer la valeur de `momentum`


In [ ]:
image_size = 176
batch_size = 512
lr = 2.
weight_decay = 5e-4
momentum = 0.9

### 2. (Optionnel) : Ajouter de la Data Augmentation

**TODO Optionnel :** 
Vous pouvez aussi choisir d'ajouter de la *Data Augmentation* dans `dlojz_imagenetrace.py` comme dans le TP de ce matin.

* RandAugment
* MixUp
* CutMix
* Autres ...

**Remarque** : Si la *Data Augmentation* permet d'atteindre des *scores* de métrique plus élevés, il faudra normalement plus d'*epochs* pour l'atteindre, l'apprentissage sera plus long. Il est donc nécessaire de prévoir une descente de gradient plus agréssive en adaptant la taille d'image, le *batch size*, le *learning rate* et l'*optimizer* ou de prévoir d'utiliser un modèle qui apprend plus vite.

### 3. (Optionnel) : Changer de modèle

Vous pouvez aussi choisir de changer de modèle.

Par exemple, en choisissant un modèle `torchvision` :

In [ ]:
import torchvision.models as models
print([
    k for k, v in models.__dict__.items()
    if callable(v) and k[0].islower() and k[0] != "_"
])

In [ ]:
model = models.wide_resnet50_2()

In [ ]:
print('number of total parameters: {}'.format(sum([p.numel() for p in model.parameters()])))
print('number of trainable parameters: {}'.format(sum([p.numel() for p in model.parameters() if p.requires_grad])))

Par exemple, en choisissant un modèle `timm` :

`*resnet*` pour avoir une liste des modèles dont le nom comporte `resnet`. Vous pouvez faire une recherche différente. 

In [ ]:
import timm
timm.list_models('*resnet*')

In [ ]:
model = timm.create_model('seresnet50t')

In [ ]:
print('number of total parameters: {}'.format(sum([p.numel() for p in model.parameters()])))
print('number of trainable parameters: {}'.format(sum([p.numel() for p in model.parameters() if p.requires_grad])))

**TODO Optionnel :**

Si vous choisissez de changer de modèle, il faudra dans `dlojz_imagenetrace.py` :

* importer `timm` si vous utilisez la librairie :
```python
import timm
```

* charger le modèle choisi et décrire le modèle dans `archi_model` pour les *log WeightandBiases*.

par exemple : 

```python
model = models.wide_resnet50_2()
model = model.to(gpu)

archi_model = 'Wide Resnet-50 2'

```

ou

```python
model = timm.create_model('seresnet50t')
model = model.to(gpu)

archi_model = 'SE Resnet-50 t'

```

### Tester votre solution

In [ ]:
command = f'dlojz_imagenetrace.py -b {batch_size} --image-size {image_size} --lr {lr} --wd {weight_decay} --mom {momentum}  --test'
print(command)

#### Optionnel : paramètres d'optimisation du DataLoader

Si vous souhaitez appliquer des paramètres du DataLoader différents des paramètres par défaut, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code` et la modifier.

--------------------

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

In [ ]:
display_slurm_queue(name)

In [ ]:
controle_technique(jobid)

In [ ]:
turbo_profiler(jobid)

![Commentaires](images/cedez.png "Attention une fois sûr de votre solution, vous pouvez lancer un apprentissage complet qui tourbera la nuit")

---------------------------

## Apprentissage complet sur 32 GPU (à lancer en toute fin de journée)

![race](./images/F1.png)

> "L'important dans la vie, ce n'est point le triomphe, mais le combat. L'essentiel n'est pas d'avoir vaincu, mais de s'être bien battu."  -- Pierre de Coubertin


**TODO :** Une fois que vous avez choisi la configuration que vous souhaitez engager pour la course, la fonction suivante permet de générer la bonne commande à soumettre à *SLURM* avec le bon nombre d'*epochs*, les bonnes configurations de *taille de batch par GPU*  et de *taille d'image*, à condition d'avoir fourni le bon `jobid`.

In [ ]:
command = imagenet_starter(jobid, jour2=True, lr=lr, moment=momentum, weight_decay=weight_decay)
assert command.split()[0] == 'dlojz_imagenetrace.py', "Veuillez bien mettre l'option jour2=True, svp !!" 
command

#### Optionnel : paramètres d'optimisation du DataLoader

Si vous souhaitez appliquer des paramètres du DataLoader différents des paramètres par défaut, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code` et la modifier.

-----------------

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobid = ['xxxxx']` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.


In [ ]:
#jobid = ['91607']

### Visualisation des résultats

In [ ]:
jobids = ['1494173', jobid[0]]

In [ ]:
plot_accuracy(jobids[:1])

In [ ]:
plot_time(jobids[:1])

In [ ]:
display_slurm_queue(name+'_race')

#### Votre résultat

In [ ]:
plot_accuracy(jobids)

In [ ]:
plot_time(jobid)

### Publication des Résultats sur WandB

Décommenter la ligne `#!wandb sync --sync-all` pour publier les résultats sur le dépôt WandB

In [ ]:
import os
os.environ['WANDB_API_KEY']='2ecf1cc3a3fe45c17b480e66dd0f390c85763d42'
#!wandb sync --sync-all

https://wandb.ai/dlojz/Imagenet%20Race%20Cup?workspace=user-bcabot

--------------